## <font color='seagreen'> Notebook1 Plotting Fluorescence Distributions of Vybrant DiD-MSm-DsRed intensities </font>

In this notebook, we will plot the intensity distributions of both the Vybrant DiD dye (CY5 channel) and Msm-DSRed (CY3 channel) intensity of single-bacteria during macrophage infections. 

In [ ]:
#Load libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import scipy as sp
import numpy as np

***

## Load Data

Load single bacteria frame.

In [ ]:
big_frame=pd.read_pickle('../Results/final_single_bacteria.pkl')
big_frame=big_frame.reset_index()

In [ ]:
big_frame.head(3)

***

## Plotting Data

Plot unfiltered Fluorescence distributions (shown in **Figure 3D**).

In [ ]:
palettes=[sns.cubehelix_palette(light=.8, dark=0.4), sns.cubehelix_palette(start=2, rot=0, dark=0.4, light=.8)]
order=[0,12,24,36,48,72]

channels=['CY5','CY3']
i=0
for channel in channels:
    axis = sns.violinplot(x="time", y=f"{channel}_intensity_mean", data=big_frame, color=".8", scale='width',order=order)
    axis=sns.stripplot(x='time', y=f'{channel}_intensity_mean',data=big_frame, palette=palettes[i],alpha=0.8, s=3, edgecolor='black', linewidth=.2, order=order)
    axis.set_xlabel('time (hours)')
    axis.set_title(f'{channel} Fluorescence intensity over time (A.U)')
    axis.set_ylabel('Fluorescence intensity (A.U)')
    #uncomment to overwrite and save figure
    #plt.savefig(fr'..\Figures\unfiltered_{channel}_violinplot_decrease_over_time.svg',dpi=300)
    plt.show()
    i+=1

***

## Fitting an exponential decay function

Fit a exponential decay function through the CY5 intensity distributions over time (shown in **Figure 3C**.)

In [ ]:
def model_func(t, A, K, C):
    return A * np.exp(-K * t) + C

def partial_func(t, A, K):
    return A * np.exp(-K * t) 

def fit_exp_nonlinear(t, y):
    prem_parms, parm_cov = curve_fit(partial_func, t, y, maxfev=10000) 
    print(prem_parms)
    A, K = prem_parms
    
    opt_parms, parm_cov = curve_fit(model_func, t, y, p0=(A, K, 0))
    A, K, C = opt_parms
    
    return A, K, C

t = np.array([0,12,24,36,48,72])

y= [big_frame[big_frame['time']==hour].CY5_intensity_mean.mean() for hour in t]

def plot(ax, t, y, fit_y, fit_parms):
    A, K, C = fit_parms
    ax.plot(t, fit_y, 'b-',
      label='Fitted Function:\n $y = %0.2f e^{%0.2f t} + %0.2f$' % (A, K, C))
    ax.legend(bbox_to_anchor=(1.0, .98), fancybox=True, shadow=True)
    ax.scatter(t,y)

fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)


 # Non-linear Fit
A, K, C = fit_exp_nonlinear(t, y)
fit_y = model_func(t, A, K, C)
plot(ax1, t, y,fit_y, (A, K, C))
ax1.set_title('Non-linear Fit')

#uncomment to overwrite and save figure
#plt.savefig('..\Figures\exponential_decay_fit.png')

***

## Plotting Intensity distribution over time

Changes in Intensity distributions over time.

In [ ]:
channels=['CY5','CY3']
i=0
order=[0,12,24,36,48,72]


for channel in channels:
    sns.histplot(data=big_frame, x=f"{channel}_intensity_mean", element="bars",hue='time',palette='colorblind' ,multiple='dodge',hue_order=order)
    axis.set_xlabel('time (hours)')
    axis.set_title(f'{channel} Fluorescence intensity over time (A.U)')
    axis.set_ylabel('Fluorescence intensity (A.U)')
    #uncomment to overwrite and save figure
    #plt.savefig(fr'..\Figures\{channel}_barplot_decrease_over_time.svg',dpi=300)
    plt.show()
    i+=1

Bargraphs split.

In [ ]:
g = sns.FacetGrid(big_frame, col="time")
g.map(sns.histplot, "CY3_intensity_mean", binwidth=1000, color='green')

#uncomment to overwrite and save figure
plt.savefig(r'..\Figures\CY3_intensity_over_time_several_barplots.svg')

In [ ]:
g = sns.FacetGrid(big_frame, col="time")
g.map(sns.histplot, "CY5_intensity_mean", binwidth=1000, color='purple')

#uncomment to overwrite and save figure
#plt.savefig(r'..\Figures\CY5_intensity_over_time_several_barplots.svg')

***

In the next Notebook (**Notebooks_Data_exploration\Notebook2_Plotting_Fraction_of_Positive_Vybrant_DiD_and_MSm-DsRed_Bacteria.ipynb**), we will determine the fraction of Vybrant Did bacteria in the entire viable bacterial population.